# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
#Important weather data CSV file from Weather Py
weather_data = pd.read_csv("cities.csv")
weather_data_df = weather_data
weather_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [8]:
#Find the number of nan values
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     548 non-null    int64  
 1   City        548 non-null    object 
 2   Cloudiness  548 non-null    int64  
 3   Country     544 non-null    object 
 4   Date        548 non-null    int64  
 5   Humidity    548 non-null    int64  
 6   Lat         548 non-null    float64
 7   Lng         548 non-null    float64
 8   Max Temp    548 non-null    float64
 9   Wind Speed  548 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 42.9+ KB


In [9]:
#Modify weather_data to remove the nan values
weather_data_df=weather_data_df[~weather_data_df['Country'].isna()]

In [10]:
#Confirm the same amount of rows in each column and nan values have been dropped
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 0 to 547
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     544 non-null    int64  
 1   City        544 non-null    object 
 2   Cloudiness  544 non-null    int64  
 3   Country     544 non-null    object 
 4   Date        544 non-null    int64  
 5   Humidity    544 non-null    int64  
 6   Lat         544 non-null    float64
 7   Lng         544 non-null    float64
 8   Max Temp    544 non-null    float64
 9   Wind Speed  544 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 46.8+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#configure g-maps
gmaps.configure(api_key=g_key)

In [12]:
#create a dataframes for countries
countries = list(weather_data_df["Country"].unique())
countries

['RU',
 'MU',
 'GL',
 'CN',
 'ZA',
 'PF',
 'AU',
 'CL',
 'CK',
 'MX',
 'US',
 'AR',
 'BD',
 'FR',
 'BR',
 'TH',
 'IN',
 'CA',
 'NO',
 'TR',
 'PY',
 'EC',
 'NI',
 'MG',
 'PH',
 'IR',
 'PE',
 'IS',
 'ID',
 'LK',
 'PT',
 'ES',
 'KI',
 'SO',
 'GR',
 'IT',
 'NP',
 'NZ',
 'OM',
 'ML',
 'SA',
 'JP',
 'GA',
 'SL',
 'NG',
 'MV',
 'NC',
 'LA',
 'CV',
 'UY',
 'CO',
 'DZ',
 'RW',
 'GY',
 'AZ',
 'DO',
 'GF',
 'PG',
 'HN',
 'MA',
 'MQ',
 'LR',
 'UG',
 'AO',
 'IE',
 'PL',
 'CD',
 'MZ',
 'UA',
 'PK',
 'ET',
 'FI',
 'RE',
 'LY',
 'VE',
 'KY',
 'TZ',
 'CU',
 'TT',
 'GH',
 'MM',
 'NU',
 'GM',
 'TL',
 'TM',
 'MN',
 'SB',
 'BF',
 'ER',
 'TC',
 'VN',
 'TD',
 'JM',
 'VG',
 'CM',
 'ZW',
 'KZ',
 'RO',
 'BO',
 'ZM',
 'AE',
 'BN',
 'BB',
 'IQ',
 'CI',
 'VU',
 'DE',
 'SK',
 'GB',
 'QA',
 'MK',
 'KR',
 'AW',
 'KE',
 'CF',
 'CH',
 'MR',
 'NE',
 'BS',
 'BY',
 'YE',
 'SN',
 'SY']

In [14]:
#Use the Lat and Lng as locations and Humidity as the weight.
countries = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"]
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(countries, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#create new dataframe
new_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)
wind_speed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0
new_df[max_temp& wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
70,ponta do sol,73.68,5.32,0
88,cidreira,76.56,5.50,0
138,morondava,79.08,9.06,0
139,vaini,78.90,1.59,0
176,nador,75.20,6.93,0
297,mogok,72.24,2.77,0
444,birjand,71.60,9.17,0


In [16]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 0 to 547
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        544 non-null    object 
 1   Max Temp    544 non-null    float64
 2   Wind Speed  544 non-null    float64
 3   Cloudiness  544 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 21.2+ KB


In [7]:
# Find the nan values
weather_data_df[weather_data_df['Country'].isna()]

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
285,285,luderitz,39,NaN,1558378806,72,-26.65,15.16,62.60,18.34
446,446,henties bay,0,NaN,1558378880,91,-22.12,14.28,59.00,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [18]:
#Set parameters to search for hotels with 5000 meters
#Hit the Google Places API for each city's coordinates
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))